In [ ]:
!pip install llama-index
!pip install openai
!pip install deepeval
!pip install llama-index.readers.json

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.9/335.9 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Preparing metadata (setup.py) ... done
     ━━━━━━

In [ ]:
## Simple Baseline RAG

from llama_index.core import VectorStoreIndex, Response
from llama_index.readers.json import JSONReader
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from google.colab import userdata

import pandas as pd
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

Settings.llm = OpenAI(temperature=0.2, model="gpt-3.5-turbo")

# Set default chunk size and overlap
Settings.chunk_size = 1024
Settings.chunk_overlap = 20

reader = JSONReader(
    # The number of levels to go back in the JSON tree. Set to 0 to traverse all levels. Default is None.
    levels_back=0,
)

flexsteel_doc = reader.load_data(input_file="/content/0000037472-23-000024.json", extra_info={})

flexsteel_index = VectorStoreIndex.from_documents(flexsteel_doc)
flexsteel_engine = flexsteel_index.as_query_engine(similarity_top_k=3)

query_engine_tools = [
    QueryEngineTool(
        query_engine=flexsteel_engine,
        metadata=ToolMetadata(
            name="flexsteel_10k",
            description=(
                "Provides information about flexsteel financials for year 2023"
            ),
        ),
    )
]


user_input = "What is the net sales of flexsteel comapny in residential area in 2023"

response_object = flexsteel_engine.query(
    user_input
)
print(response_object)






$393,692


In [ ]:
# Simple Evaluator
filepath = '/content/evaluation_results4.xlsx'
df = pd.read_excel(filepath, sheet_name= 'kg_rag')
queries = df['Query']
responses = df['Response']
sources = df['Source']

import deepeval
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.metrics import FaithfulnessMetric
from deepeval.test_case import LLMTestCase
from deepeval import evaluate

deepeval.login_with_confident_api_key(userdata.get('DEEPEVAL_API_KEY'))


metric1 = AnswerRelevancyMetric(
    threshold=0.5,
    model="gpt-3.5-turbo",
    include_reason=True
)


metric2 = FaithfulnessMetric(
    threshold=0.5,
    model="gpt-3.5-turbo",
    include_reason=True
)

lst  =[]
for query, response, source in zip(queries, responses, sources):
        lst.append(LLMTestCase(
            input= query,
            actual_output= response,
            retrieval_context= [source]
        ))

from deepeval.dataset import EvaluationDataset

# # Initialize empty dataset object
# dataset = EvaluationDataset(lst)

# # Pull from Confident
# dataset.push(alias="My Confident Dataset")

# dataset.pull(alias ="My Confident Dataset")
# dataset.evaluate([metric1 ,metric2])